In [1]:
import numpy as np
import tensorflow as tf
from csv import reader
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from pymystem3 import Mystem
import regex
% matplotlib inline 

/home/gavrilovilya/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
Model = tf.contrib.keras.models.Model
Sequential = tf.contrib.keras.models.Sequential
Conv1D = tf.contrib.keras.layers.Conv1D
Dense = tf.contrib.keras.layers.Dense
MaxPool1D = tf.contrib.keras.layers.MaxPool1D
GlobalMaxPool1D = tf.contrib.keras.layers.GlobalMaxPool1D
GlobalMaxPool2D = tf.contrib.keras.layers.GlobalMaxPool2D
Input = tf.contrib.keras.layers.Input
Concat = tf.contrib.keras.layers.concatenate
Dropout = tf.contrib.keras.layers.Dropout
Embedding = tf.contrib.keras.layers.Embedding
GRU = tf.contrib.keras.layers.GRU
#TimeDistributed = tf.contrib.keras.layers.TimeDistributed

Adam = tf.contrib.keras.optimizers.Adam
RMSprop = tf.contrib.keras.optimizers.RMSprop
SGD = tf.contrib.keras.optimizers.SGD

In [3]:
dataset = pd.read_csv('dataset1.csv')

In [4]:
dataset

,Unnamed: 0,PRODUCT,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Что особенно ценно - при профилактике принтера...,23.10.16 22:26:00,5.0,"хорошая скорость печати, двусторонняя печать, ...","Рядом ""на стол"" не поставишь, место много зани...",1.0,2.0,0.0
1,1,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Он у нас три месяца в офисе стоит уже - большо...,17.04.16 11:17:42,5.0,NaN,NaN,NaN,0.0,2.0
2,2,30024725,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,"Просто небо и земля с тем принтером, который у...",15.04.16 20:22:47,4.0,NaN,NaN,NaN,0.0,1.0
3,3,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,С такой вроде бы навороченной техникой все смо...,У нас в кабинете недавно его поставили. Первое...,18.04.16 20:01:57,5.0,NaN,NaN,NaN,0.0,0.0
4,4,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Очень многофункциональная штука. Есть все нео...,23.04.16 12:03:39,5.0,NaN,NaN,NaN,1.0,0.0
5,5,30024728,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Производительное и скоростное лазерное МФУ. В ...,31.05.16 11:35:09,5.0,NaN,NaN,NaN,0.0,0.0
6,6,30024729,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,Отличный по функционалу агрегат!,Скорость печати у него приличная - предшествен...,14.04.16 16:23:11,5.0,NaN,NaN,NaN,1.0,0.0
7,7,30024729,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,Мнение эксперта,Цветное МФУ от Kyocera позволит печатать в цве...,28.05.16 12:37:39,5.0,NaN,NaN,NaN,1.0,0.0
8,8,30024729,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Отличное МФУ для как для офиса тапк и для дома...,29.05.16 14:25:57,5.0,NaN,NaN,NaN,0.0,0.0
9,9,30024730,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Купил этот принтер для печати цветных фотограф...,23.04.16 12:04:19,5.0,NaN,NaN,NaN,1.0,0.0


In [5]:
dataset[~pd.isnull(dataset['BENEFITS']) | ~pd.isnull(dataset['DRAWBACKS'])]

,Unnamed: 0,PRODUCT,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Что особенно ценно - при профилактике принтера...,23.10.16 22:26:00,5.0,"хорошая скорость печати, двусторонняя печать, ...","Рядом ""на стол"" не поставишь, место много зани...",1.0,2.0,0.0
23,23,30024742,5150101,1,HEADPHONES,Sony,REVIEW,NaN,"Уже второй WALKMAN, которым пользуюсь с больши...",03.07.17 14:11:47,5.0,"Нет проводов, лёгкий, портативный, заряжается,...","Нет блокировки кнопок, как на ранних моделях",1.0,NaN,NaN
33,33,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,"Телефон просто шикарный, покупал его для 4G и...",04.11.16 15:26:23,5.0,Соотношение цена качество и вообще их много,Их просто нет,1.0,5.0,3.0
34,34,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,"Пользуюсь 7 месяцев.В принципе доволен,рабочая...",29.01.17 22:27:26,3.3,"Низкая цена,для своих характеристик.Хороший яр...",Долго заряжается батарея и быстро разряжается.,1.0,8.0,3.0
35,35,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,Смотрю тут много восторженных отзывов после од...,24.01.17 23:10:28,1.0,Их нет,"очень многое,самое главное слабая батарея",0.0,7.0,18.0
37,37,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,Пользуюсь данным смартфоном уже больше полгод...,20.01.17 02:38:54,5.0,"Хорошая связь, быстрый интернет, хороший дисп...",Нет подсветки нижних боковых сенсорных функцио...,1.0,5.0,0.0
38,38,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,Когда начал умерать мой предыдущий смартфон са...,04.01.17 14:50:48,5.0,NaN,Не подключается USB флэшка,1.0,1.0,1.0
39,39,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,Полный минус,03.01.17 14:10:52,2.0,Нет,"Пропадает связь, мало встроенной памяти, камер...",0.0,1.0,4.0
40,40,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,"Только купил, сразу же понравился красивый ярк...",08.01.17 18:03:19,5.0,"Начинка очень хорошая, игрушки летаю. Камера н...",минусов пока не нашел,1.0,1.0,0.0
42,42,30024751,2050104,13,SAMSUNG SMARTPHONES,Samsung,REVIEW,NaN,Супер,02.01.17 00:27:28,5.0,все,Пока нет,1.0,4.0,0.0


In [6]:
def encoded(l, w2n):
    for j in range(len(l)):
        l[j] = w2n.get(l[j], 0)
    return l

In [7]:
def encoded_all(text_comment, text_pcomment, text_ncomment, 
                word2num_comment):
    encoded_comment = []
    encoded_pcomment = []
    encoded_ncomment = []
    
    for i in tqdm(range(len(text_comment))):
        encoded_comment.append(encoded(text_comment[i], word2num_comment))
        encoded_pcomment.append(encoded(text_pcomment[i], word2num_comment))
        encoded_ncomment.append(encoded(text_ncomment[i], word2num_comment))
        
    return {'c' : encoded_comment, 'p' : encoded_pcomment, 'n' : encoded_ncomment}

In [8]:
def get_prepared_comment_trainig(data):
    text_comment = []
    text_pcomment = []
    text_ncomment = []
    
    set_words_comment = set()
    
    for line in tqdm(data.iterrows()):
        if type(line[1]['DRAWBACKS']) is str or type(line[1]['BENEFITS']) is str:
            comment = line[1]['TEXT'].rstrip('\n')
            ncomment = '' if not type(line[1]['DRAWBACKS']) is str else line[1]['DRAWBACKS'].rstrip('\n')
            pcomment = '' if not type(line[1]['BENEFITS']) is str else line[1]['BENEFITS'].rstrip('\n')
            set_words_comment.update(comment.split())
            text_comment.append(regex.sub('[^\p{Cyrillic}]', ' ', comment.lower()).split())
            text_pcomment.append(regex.sub('[^\p{Cyrillic}]', ' ', pcomment.lower()).split())
            text_ncomment.append(regex.sub('[^\p{Cyrillic}]', ' ', ncomment.lower()).split())
    
    word2num_comment = dict((word, num + 2) for num, word in enumerate(set_words_comment))
    
    comments = encoded_all(text_comment, text_pcomment, text_ncomment, word2num_comment)
    
    return comments, word2num_comment

In [9]:
def get_prepared_comment_test(data, word2num_comment):
    text_comment = []
    text_pcomment = []
    text_ncomment = []
     
    for line in tqdm(data.iterrows()):
        if type(line[1]['DRAWBACKS']) is str or type(line[1]['BENEFITS']) is str:
            comment = line[1]['TEXT'].rstrip('\n')
            ncomment = '' if not type(line[1]['DRAWBACKS']) is str else line[1]['DRAWBACKS'].rstrip('\n')
            pcomment = '' if not type(line[1]['BENEFITS']) is str else line[1]['BENEFITS'].rstrip('\n')
            text_comment.append(comment.split())
            text_pcomment.append(pcomment.split())
            text_ncomment.append(ncomment.split())
            
    return encoded_all(text_comment, text_pcomment, text_ncomment, 
                       word2num_comment)

In [10]:
def prepare_data(start_data):
    data = dataset[~pd.isnull(dataset['BENEFITS']) | ~pd.isnull(dataset['DRAWBACKS'])]
    data_train, data_test = train_test_split(data, test_size=0.1, random_state=42)
    data_train, data_val = train_test_split(data_train, test_size=0.1, random_state=42)
    
    train_comments, w2n = get_prepared_comment_trainig(data_train)
    test_comments = get_prepared_comment_test(data_test, w2n)
    val_comments = get_prepared_comment_test(data_val, w2n)

    return {
        'train' : to_slices(train_comments),
        'test' : to_slices(test_comments),
        'val' : to_slices(val_comments),
        'w2n' : w2n
    }

In [11]:
def window(comment_old, add_comment, list_com, list_tar):
    full = comment_old + add_comment + [1]
    prev_i = 0
    for i in range(len(comment_old) + 1, len(full)):
        list_com.append(full[prev_i : i])
        list_tar.append(full[i])
        prev_i += 1

In [12]:
def to_slices(data):
    final_data = []
    final_target = []
    
    for i in range(len(data['c'])):
        window(data['c'][i], data['p'][i], final_data, final_target)
#        window(data['c'][i], data['n'][i], final_data, final_target)
    return final_data, final_target

In [13]:
def ohe(vals, max_len):
    enc = np.zeros((len(vals), max_len))
    enc[np.arange(len(vals), dtype=int), vals] = 1
    return enc

In [14]:
def save_to_ssd(data, batch_size, max_len, num_words, type_ = 'train'):
    prev_b = 0
    for num_b in tqdm(range(batch_size, len(data[0]), batch_size)):
        x = tf.contrib.keras.preprocessing.sequence.pad_sequences(data[0][prev_b : num_b], maxlen=max_len, 
                                                                 padding='pre', truncating='pre')
        y = ohe(data[1][prev_b : num_b], num_words)
        np.save('/kali/data/{}_x_{}'.format(type_, num_b), x)
        np.save('/kali/data/y_{}'.format(type_, num_b), y)

In [15]:
def generator(data, batch_size, max_len, num_words):
    while True:
        prev_b = 0
        for num_b in range(batch_size, len(data[0]), batch_size):
            x = tf.contrib.keras.preprocessing.sequence.pad_sequences(data[0][prev_b : num_b], maxlen=max_len, 
                                                                     padding='pre', truncating='pre')
            y = ohe(data[1][prev_b : num_b], num_words)
            prev_b = num_b
            yield x, y

In [16]:
prepare_data = prepare_data(dataset)

43743it [00:07, 5711.17it/s]
100%|██████████| 43743/43743 [00:00<00:00, 44796.87it/s]
5401it [00:00, 8781.50it/s]
100%|██████████| 5401/5401 [00:00<00:00, 36291.54it/s]
4861it [00:00, 8850.38it/s]
100%|██████████| 4861/4861 [00:00<00:00, 38865.32it/s]


In [17]:
def get_ans(probs):
    return np.random.choice(np.arange(len(probs)), size=1, p = probs)

In [18]:
num_words = len(prepare_data['w2n'])
num_words

225405

In [19]:
max_len = 0
for i in range(len(prepare_data['train'][0])):
    if len(prepare_data['train'][0][i]) > max_len:
        max_len = len(prepare_data['train'][0][i])
max_len

360

In [20]:
batch_size = 128
train_gen = generator(prepare_data['train'], batch_size, max_len, len(prepare_data['w2n']))
test_gen = generator(prepare_data['test'], batch_size, max_len, len(prepare_data['w2n']))
val_gen = generator(prepare_data['val'], batch_size, max_len, len(prepare_data['w2n']))

In [21]:
# %%time
# save_to_ssd(prepare_data['train'], batch_size, max_len, len(prepare_data['w2n']), 'train')

In [22]:
def get_net_next_word(input_shape, num_words, input_length, emb_size=300, units_GRU=300, optimizer=RMSprop(), 
                      prob_drop=0.3):
    input_text = Input(shape = input_shape, dtype=tf.float32)
    emb = Embedding(input_dim=num_words + 1, output_dim=emb_size, input_length=input_length)(input_text)
    dropout1 = Dropout(prob_drop)(emb)

    gru1 = GRU(units_GRU)(dropout1)
    dropout2 = Dropout(prob_drop)(gru1)
    
    dense = Dense(units=num_words, activation='softmax')(dropout2)
    
    model = Model(inputs=input_text, outputs=dense)
    model.compile(optimizer, 'categorical_crossentropy')
    
#     input_other = Input(shape = (2920,))
#     concat = Concat([dropout, input_other])
    
#     dense = Dense(units=5, activation='softmax')(concat)
#     model = Model(inputs=(input_text, input_other), outputs=dense)
#     model.compile(optimizer, 'categorical_crossentropy')
    
    return model

In [23]:
net2 = get_net_next_word(input_shape=(max_len,), num_words=num_words, input_length=max_len)

In [24]:
net2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 360)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 360, 300)          67621800  
_________________________________________________________________
dropout_1 (Dropout)          (None, 360, 300)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               540900    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 225405)            67846905  
Total params: 136,009,605
Trainable params: 136,009,605
Non-trainable params: 0
______________________________________________________________

In [ ]:
net2.fit_generator(train_gen, len(prepare_data['train'][0]) // batch_size, epochs=1,
                  validation_data=test_gen, validation_steps=len(prepare_data['test'][0]) // batch_size)

Epoch 1/1
 319/3409 [=>............................] - ETA: 1449s - loss: 8.1500 

In [25]:
n2w = dict((i, k) for k, i in prepare_data['w2n'].items())

In [26]:
t1 = next(train_gen)

In [27]:
p = net2.predict(t1[0])

In [28]:
p.shape

(128, 225405)

In [29]:
ans = np.zeros(batch_size)
for i in range(batch_size):
    ans[i] = get_ans(p[i])

In [30]:
#ans = np.argmax(p, axis=1)
#ans

In [31]:
for i in range(len(t1[0])):
    for j in range(len(t1[0][i])):
        print(n2w.get(t1[0][i][j], ''), end=' ')
    print('Predict: ', end= ' ')
    print(n2w.get(ans[i], 'unknown'))

                                                                                                                                                                                                                                                                                     так как выбора товара в мвидео  нет абсолютно и изначально я хотел взять и уже собирался оплачивать как приглядевшись понял что она не механическая окончательно разочаровавшись в зелёных стал смотреть другие и так как выбора нет пришлось взять и вы знаете я не расстроен три памяти макроса с шестью клавишами две регулировки подсветки общая и только игровые клавиши отдельная мультимедийная панель свитчи для наушников и микрофона настройка макросов производится через по клавиатуры и там же можно настроить подсветку логотипа под себя механическая Predict:  плоскости
                                                                                                                                                          

In [5]:
#pd.read_csv('dataset2.csv')

In [6]:
# def get_prepared_text_trainig(data, len_win):
#     text = []
#     set_words = set()
# #    m = Mystem()
#     for line in tqdm(data.iterrows()):
#         comment = line[1]['TEXT'].rstrip('\n')
#         ncomment = '' if not type(line[1]['DRAWBACKS']) is str else line[1]['DRAWBACKS'].rstrip('\n')
#         pcomment = '' if not type(line[1]['BENEFITS']) is str else line[1]['BENEFITS'].rstrip('\n')
#         general_comment = comment + ' ' + ncomment + ' ' + pcomment
# #        general_comment = ' '.join([word for word in m.lemmatize(general_comment.lower())])
#         text.append(general_comment)
#         set_words.update(general_comment.split())
    
#     word2num = dict((word, num + 2) for num, word in enumerate(set_words))
#     print (type(word2num))
#     encoded_text = []
#     target_word = []
#     for j, comment in tqdm(enumerate(text)):
#         comment = comment.split()
#         encoded_comment = np.ones(len(comment) + 1)
#         for i in range(len(comment)):
#             encoded_comment[i] = word2num.get(comment[i], 0)
#         encoded_comment = np.concatenate([np.zeros(len_win - 1), encoded_comment])
#         prev_slide = 0
#         for num_slide in range(len_win, len(encoded_comment)):
#             encoded_text.append(encoded_comment[prev_slide : num_slide])
#             prev_slide += 1
#             target_word.append(encoded_comment[num_slide])
    
#     return np.array(encoded_text), np.array(target_word), word2num

# def get_prepared_text_test(data, len_win, word2num):
#     text = []
# #    m = Mystem()
#     for line in tqdm(data.iterrows()):
#         comment = line[1]['TEXT'].rstrip('\n')
#         ncomment = '' if not type(line[1]['DRAWBACKS']) is str else line[1]['DRAWBACKS'].rstrip('\n')
#         pcomment = '' if not type(line[1]['BENEFITS']) is str else line[1]['BENEFITS'].rstrip('\n')
#         general_comment = comment + ' ' + ncomment + ' ' + pcomment
# #        general_comment = ' '.join([word for word in m.lemmatize(general_comment.lower())])
#         text.append(general_comment)
    
#     encoded_text = []
#     target_word = []
#     for j, comment in tqdm(enumerate(text)):
#         comment = comment.split()
#         encoded_comment = np.ones(len(comment) + 1)
#         for i in range(len(comment)):
#             encoded_comment[i] = word2num.get(comment[i], 0)
# #         remains = (len(encoded_comment) - 1) % len_win
#         encoded_comment = np.concatenate([np.zeros(len_win - 1), encoded_comment])
#         prev_slide = 0
#         for num_slide in range(len_win, len(encoded_comment)):
#             encoded_text.append(encoded_comment[prev_slide : num_slide])
#             prev_slide += 1
#             target_word.append(encoded_comment[num_slide])
    
#     return np.array(encoded_text), np.array(target_word)

In [7]:
# def prepared_data(data):
#     data_train, data_test = train_test_split(data, test_size=0.1, random_state=42)
#     data_train, data_val = train_test_split(data_train, test_size=0.1, random_state=42)
    
#     len_win = 20
#     text_train, target_train, word2num = get_prepared_text_trainig(data_train, len_win)
#     text_test, target_test = get_prepared_text_test(data_test, len_win, word2num)
#     text_val, target_val = get_prepared_text_test(data_val,  len_win, word2num)
            
#     return {'train' : (text_train, target_train),
#     'test' : (text_test, target_test),
#     'val' : (text_val, target_val)}

In [8]:
# ready_data = prepared_data(dataset)

219461it [07:29, 488.28it/s]
1321it [00:00, 13207.12it/s]

<class 'dict'>


219461it [00:15, 13914.56it/s]
27095it [00:55, 486.89it/s]
27095it [00:01, 13583.61it/s]
24385it [00:49, 489.65it/s]
24385it [00:01, 13941.18it/s]


In [9]:
# np.unique(ready_data['train'][0])

array([  0.00000000e+00,   2.00000000e+00,   3.00000000e+00, ...,
         1.18798000e+05,   1.18799000e+05,   1.18800000e+05])

In [10]:
# np.unique(ready_data['train'][1])

array([  1.00000000e+00,   2.00000000e+00,   3.00000000e+00, ...,
         1.18798000e+05,   1.18799000e+05,   1.18800000e+05])

In [11]:
# num_words = int(np.max(ready_data['train'][0]))

In [12]:
# num_words

118800

In [13]:
# def ohe(t, all_words):
#     res = np.zeros((len(t), all_words))
#     res[np.arange(len(t), dtype=int), t.astype(int)] = 1
#     return res

In [14]:
# def generator(data, target, batch_size, all_words):
#     while True:
#         prev_batch = 0
#         for batch in range(batch_size, len(data), batch_size):
#             batch_data = data[prev_batch : batch]
#             batch_target = ohe(target[prev_batch : batch], all_words)
#             prev_batch = batch
            
#             yield (batch_data, batch_target)

In [15]:
# def get_net_next_word(input_shape, num_words, input_length, emb_size=300, units_GRU=300, optimizer=RMSprop(), 
#                       prob_drop=0.3):
#     input_text = Input(shape = input_shape, dtype=tf.float32)
#     emb = Embedding(input_dim=num_words + 1, output_dim=emb_size, input_length=input_length)(input_text)
#     dropout1 = Dropout(prob_drop)(emb)

#     gru1 = GRU(units_GRU)(dropout1)
#     dropout2 = Dropout(prob_drop)(gru1)
    
#     dense = Dense(units=num_words, activation='softmax')(dropout2)
    
#     model = Model(inputs=input_text, outputs=dense)
#     model.compile(optimizer, 'categorical_crossentropy')
    
# #     input_other = Input(shape = (2920,))
# #     concat = Concat([dropout, input_other])
    
# #     dense = Dense(units=5, activation='softmax')(concat)
# #     model = Model(inputs=(input_text, input_other), outputs=dense)
# #     model.compile(optimizer, 'categorical_crossentropy')
    
#     return model

In [16]:
# ready_data['train'][0].shape

(15958635, 20)

In [17]:
net1 = get_net_next_word(input_shape=(20,), num_words=num_words, input_length=20)

In [18]:
# train_gen = generator(ready_data['train'][0], ready_data['train'][1], 32, num_words)
# test_gen = generator(ready_data['test'][0], ready_data['test'][1], 32, num_words)

In [19]:
# batch_size = 512
# net1.fit_generator(train_gen, ready_data['train'][0].shape[0] // batch_size, epochs=2,
#                   validation_data=test_gen, validation_steps=ready_data['test'][0].shape[0] // batch_size)

Epoch 1/2
  964/31169 [..............................] - ETA: 2421s - loss: 7.6009  

KeyboardInterrupt: 

In [ ]:
# net1.fit(ready_data['train'][0], ready_data['train'][1], batch_size=64, epochs=1, 
#          validation_data = (ready_data['test'][0], ready_data['test'][1]))